# Sending Traces from Langtrace to New Relic: A Step-by-Step Guide

This notebook demonstrates how to use Langtrace with OpenAI and Qdrant, and send traces to New Relic using OpenTelemetry.

## Step 1: Import Required Libraries and Initialize Clients

In [ ]:
import os
import time
import openai
from qdrant_client import QdrantClient
from langtrace_python_sdk import langtrace, with_langtrace_root_span
from typing import List, Dict, Any

# Initialize environment and clients
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"
langtrace.init(api_key='YOUR_LANGTRACE_API_KEY')
qdrant_client = QdrantClient(":memory:") 
openai_client = openai.Client(api_key=os.getenv("OPENAI_API_KEY"))

print("Libraries imported and clients initialized.")

## Step 2: Define Key Functions

### 2.1 Initialize Knowledge Base

In [ ]:
@with_langtrace_root_span("initialize_knowledge_base")
def initialize_knowledge_base(documents: List[str]) -> None:
    start_time = time.time()
    qdrant_client.add(
        collection_name="knowledge-base",
        documents=documents
    )
    end_time = time.time()
    print(f"Knowledge base initialized with {len(documents)} documents in {end_time - start_time:.2f} seconds")

print("Knowledge base initialization function defined.")

### 2.2 Query Vector Database

In [ ]:
@with_langtrace_root_span("query_vector_db")
def query_vector_db(question: str, n_points: int = 3) -> List[Dict[str, Any]]:
    start_time = time.time()
    results = qdrant_client.query(
        collection_name="knowledge-base",
        query_text=question,
        limit=n_points,
    )
    end_time = time.time()
    print(f"Vector DB queried in {end_time - start_time:.2f} seconds, returned {len(results)} results")
    return results

print("Vector database query function defined.")

### 2.3 Generate LLM Response

In [ ]:
@with_langtrace_root_span("generate_llm_response")
def generate_llm_response(prompt: str, model: str = "gpt-3.5-turbo") -> str:
    start_time = time.time()
    completion = openai_client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": prompt},
        ],
        timeout=10.0,
    )
    end_time = time.time()
    response = completion.choices[0].message.content
    print(f"LLM response generated in {end_time - start_time:.2f} seconds")
    return response

print("LLM response generation function defined.")

### 2.4 Implement RAG (Retrieval-Augmented Generation)

In [ ]:
@with_langtrace_root_span("rag")
def rag(question: str, n_points: int = 3) -> str:
    print(f"Processing RAG for question: {question}")
    
    context_start = time.time()
    context = "\n".join([r.document for r in query_vector_db(question, n_points)])
    context_end = time.time()
    print(f"Context retrieved in {context_end - context_start:.2f} seconds")
    
    prompt_start = time.time()
    metaprompt = f"""
    You are a software architect.
    Answer the following question using the provided context.
    If you can't find the answer, do not pretend you know it, but answer "I don't know".

    Question: {question.strip()}

    Context:
    {context.strip()}

    Answer:
    """
    prompt_end = time.time()
    print(f"Prompt constructed in {prompt_end - prompt_start:.2f} seconds")
    
    answer = generate_llm_response(metaprompt)
    print(f"RAG completed, answer length: {len(answer)} characters")
    return answer

print("RAG function defined.")

## Step 3: Demonstrate Different Queries

In [ ]:
def demonstrate_different_queries():
    questions = [
        "What is Qdrant used for?",
        "How does Docker help developers?",
        "What is the purpose of MySQL?",
        "Can you explain what FastAPI is?",
    ]
    for question in questions:
        try:
            answer = rag(question)
            print(f"Question: {question}")
            print(f"Answer: {answer}\n")
        except Exception as e:
            print(f"Error processing question '{question}': {str(e)}\n")

print("Query demonstration function defined.")

## Step 4: Initialize Knowledge Base and Run Queries

In [ ]:
# Initialize knowledge base
documents = [
    "Qdrant is a vector database & vector similarity search engine. It deploys as an API service providing search for the nearest high-dimensional vectors. With Qdrant, embeddings or neural network encoders can be turned into full-fledged applications for matching, searching, recommending, and much more!",
    "Docker helps developers build, share, and run applications anywhere — without tedious environment configuration or management.",
    "PyTorch is a machine learning framework based on the Torch library, used for applications such as computer vision and natural language processing.",
    "MySQL is an open-source relational database management system (RDBMS). A relational database organizes data into one or more data tables in which data may be related to each other; these relations help structure the data. SQL is a language that programmers use to create, modify and extract data from the relational database, as well as control user access to the database.",
    "NGINX is a free, open-source, high-performance HTTP server and reverse proxy, as well as an IMAP/POP3 proxy server. NGINX is known for its high performance, stability, rich feature set, simple configuration, and low resource consumption.",
    "FastAPI is a modern, fast (high-performance), web framework for building APIs with Python 3.7+ based on standard Python type hints.",
    "SentenceTransformers is a Python framework for state-of-the-art sentence, text and image embeddings. You can use this framework to compute sentence / text embeddings for more than 100 languages. These embeddings can then be compared e.g. with cosine-similarity to find sentences with a similar meaning. This can be useful for semantic textual similar, semantic search, or paraphrase mining.",
    "The cron command-line utility is a job scheduler on Unix-like operating systems. Users who set up and maintain software environments use cron to schedule jobs (commands or shell scripts), also known as cron jobs, to run periodically at fixed times, dates, or intervals.",
]
initialize_knowledge_base(documents)

# Run the demonstration
demonstrate_different_queries()

## Step 5: Analyze Traces

After running the queries, you can analyze the traces as follows:

1. Check the Langtrace dashboard for a visual representation of the traces.
2. Look for the 'rag' root span and its child spans to understand the flow.
3. Examine the timing information printed for each operation to identify potential bottlenecks.
4. Review any error messages printed to understand and address issues.